In [53]:
import pandas as pd
import json
import requests

from selenium import webdriver
driver = webdriver.Chrome()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


***
* 目的
    * 爬取女生配件這個分類的熱銷榜
    <font color="red"><font>
* 分析
    * Shopee網站為動態生成，經分析發
***

In [26]:
# 1st stage query: 取得項目id list
url = 'https://shopee.tw/api/v1/search_items/'
# params
param_get_list = {"by":"sales", "categoryids":2580, "limit":"50", "newest":0, "order": "desc"}
# 正確的UA才回Fetch到真正的Item list
header_get_list = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:58.0) Gecko/20100101 Firefox/58.0"} 
res = requests.get(url, params=param_get_list, headers=header_get_list)

In [34]:
# parse json response from 1st stage requests.
data = json.loads(res.text)
data

{'adjust': {'count': 10372473},
 'algorithm': '',
 'hint_keywords': [],
 'items': [{'itemid': 27596108,
   'logisticid': [43274, 56375, 36099, 46370],
   'shopid': 5212303},
  {'itemid': 262646175,
   'logisticid': [48162, 52254, 62647, 46786],
   'shopid': 9638994},
  {'itemid': 40800119,
   'logisticid': [63877, 48347, 41944, 52010, 48048, 47020],
   'shopid': 22383},
  {'itemid': 47071783,
   'logisticid': [31574, 60041, 44610, 41822],
   'shopid': 3497546},
  {'itemid': 479056620, 'logisticid': [42929, 64683], 'shopid': 18171159},
  {'itemid': 340097295, 'logisticid': [47073, 54659], 'shopid': 15222688},
  {'itemid': 143223497, 'logisticid': [46286, 48873], 'shopid': 12671360},
  {'itemid': 196401947,
   'logisticid': [58312, 39081, 32771],
   'shopid': 3143184},
  {'itemid': 4710874,
   'logisticid': [39557, 73874, 71534, 34164, 31381, 30794],
   'shopid': 22383},
  {'itemid': 50347765,
   'logisticid': [68227, 43042, 53713, 31624],
   'shopid': 2407984},
  {'itemid': 497109583, '

In [35]:
# 生成參數JSON
# 2nd stage query 使用的JSON payload 只有參數名不一樣
param2 = json.loads("{}")
param2['item_shop_ids'] = data['items']
param2 = str(param2)

In [49]:
# 2nd stage query: get item detail
url_get_item_info = 'https://shopee.tw/api/v1/items/'
header_get_item_info = {
    "Host":"shopee.tw",
    "origin":"https://shopee.tw",
    "x-csrftoken":"dDpiowxEv8u4NelUtFGUtmaPbHk5VTFD",
    "Cookie":"_ga=GA1.2.671112091.1520493824; _gid=GA1.2.620437814.1520493824; SPC_IA=-1; SPC_U=-; SPC_F=eKg1RiYkXFQJHgfnvpXVaU9Tn0K41EMG; SPC_EC=-; REC_T_ID=a2406442-22a1-11e8-a2c0-246e961721ec; SPC_T_ID=\"XEK43/UoYZi5LlA1qtXodnx4+QegAizbK2VfntFF6kbLd1oEKLS6Hrw2AkcXHrWt7P5oQrMt2HXQFAsqUDXE432Hct3HTzEVKrswGi4bQb0=\"; SPC_SI=u4o1m1n8ag1jdkr2hzcgnpeos6bxv2wc; SPC_T_IV=\"S1YMfoIGqZ0yqFif/jCQfA==\"; csrftoken=dDpiowxEv8u4NelUtFGUtmaPbHk5VTFD; cto_lwid=363964b8-6e1c-4e6d-9171-adbcf7b4a4b4; __BWfp=c1520493828173x9d2da85fc; SPC_SC_TK=; UYOMAPJWEMDGJ=; SPC_SC_UD=; _gat=1",
    "Referer":"https://shopee.tw/%E5%A5%B3%E7%94%9F%E9%85%8D%E4%BB%B6-cat.2580", # stage 1 resulting url...
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:58.0) Gecko/20100101 Firefox/58.0"
}
res2 = requests.get(url_get_item_info, json = param2, headers = header_get_item_info)
res2.text
#res2

'{"version": "99914b932bd37a50b983c5e7c90ae93b"}'